In [ ]:
!apt-get update -y
!apt-get install -y espeak-ng


!git clone https://github.com/pnnbao97/VieNeu-TTS.git
%cd VieNeu-TTS

!pip install -r requirements.txt
!pip install phonemizer # Đảm bảo thư viện này được cài

import os
os.environ["PHONEMIZER_ESPEAK_LIBRARY"] = "/usr/lib/x86_64-linux-gnu/libespeak-ng.so.1"

In [ ]:
import torch
import soundfile as sf
from vieneu_tts import VieNeuTTS
from IPython.display import Audio, display

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Đang chạy trên thiết bị: {device}")

try:
    tts = VieNeuTTS(
        backbone_repo="pnnbao-ump/VieNeu-TTS-1000h",
        backbone_device=device,
        codec_repo="neuphonic/neucodec",
        codec_device=device
    )
    print("thành công")
except Exception as e:
    print(f"Lỗi: {e}")

In [ ]:
input_text = "Xin chào, đây là thử nghiệm mô hình VieNeu TTS trên Google Colab. Tôi là dân kỹ thuật về lĩnh vực MLOps."

ref_audio_path = "./sample/id_0001.wav"
ref_text_path = "./sample/id_0001.txt"


if os.path.exists(ref_audio_path) and os.path.exists(ref_text_path):
    # Đọc text tham chiếu
    with open(ref_text_path, "r", encoding="utf-8") as f:
        ref_text_raw = f.read()

    print(f"Đang clone giọng từ: {ref_audio_path}")
    print("Đang mã hóa audio tham chiếu...")

    # Encode giọng mẫu
    ref_codes = tts.encode_reference(ref_audio_path)

    print(f"Đang tạo audio cho văn bản: '{input_text}'...")

    # Inference
    # Lưu ý: normalize_text đã được tích hợp bên trong pipeline của repo này
    wav = tts.infer(input_text, ref_codes, ref_text_raw)

    output_filename = "output_colab.wav"
    sf.write(output_filename, wav, 24000)

    print(f"Đã lưu file tại: {output_filename}")

    # Phát audio trực tiếp trên Colab
    display(Audio(output_filename, rate=24000))

else:
    print(f"Không tìm thấy file tham chiếu tại {ref_audio_path}")